## **Imports and Setting Up**

In [1]:
import pandas as pd

In [3]:
from google.colab import drive
 
#mount g drive
drive.mount('/gdrive')

# navigate to the project's folder
%cd /gdrive/'My Drive'/

Mounted at /gdrive
/gdrive/My Drive


## **Load owid-covid-data.csv into dataframe**

In [54]:
df = pd.read_csv('CSC3007_datasets/owid-covid-data.csv') 

In [5]:
pd.set_option('display.max_columns', None)

In [56]:
pd.set_option('display.max_rows', None)

In [ ]:
len(df.columns)

67

In [55]:
# replace United States with USA
df[['location']] = df[['location']].replace(['United States'], ['USA'])

## **Retrieve only the columns we need from the dataframe**

In [58]:
df = df[["continent", "location", "date", "total_cases", "new_cases", "total_deaths", "new_deaths", "total_vaccinations", "people_vaccinated", "people_fully_vaccinated", "total_boosters", "population", "population_density"]].fillna(0)

In [59]:
len(df)

197476

## **Load world_geojson.json into dataframe**

In [60]:
import json

with open('CSC3007_datasets/world_geojson.json') as data_file:    
    data = json.load(data_file)  

df_geo_world = pd.json_normalize(data, 'features', ['type'], 
                    record_prefix='features_')


In [61]:
df_geo_world.head()

,features_type,features_id,features_properties.name,features_geometry.type,features_geometry.coordinates,type
0,Feature,AFG,Afghanistan,Polygon,"[[[61.210817, 35.650072], [62.230651, 35.27066...",FeatureCollection
1,Feature,AGO,Angola,MultiPolygon,"[[[[16.326528, -5.87747], [16.57318, -6.622645...",FeatureCollection
2,Feature,ALB,Albania,Polygon,"[[[20.590247, 41.855404], [20.463175, 41.51508...",FeatureCollection
3,Feature,ARE,United Arab Emirates,Polygon,"[[[51.579519, 24.245497], [51.757441, 24.29407...",FeatureCollection
4,Feature,ARG,Argentina,MultiPolygon,"[[[[-65.5, -55.2], [-66.45, -55.25], [-66.9599...",FeatureCollection


In [62]:
df_geo_world["features_properties.name"]

0                              Afghanistan
1                                   Angola
2                                  Albania
3                     United Arab Emirates
4                                Argentina
5                                  Armenia
6                               Antarctica
7      French Southern and Antarctic Lands
8                                Australia
9                                  Austria
10                              Azerbaijan
11                                 Burundi
12                                 Belgium
13                                   Benin
14                            Burkina Faso
15                              Bangladesh
16                                Bulgaria
17                             The Bahamas
18                  Bosnia and Herzegovina
19                                 Belarus
20                                  Belize
21                                 Bolivia
22                                  Brazil
23         

## **Only get the rows based on the common values between the column:"location" from the owid-covid-data.csv and in the column:"features_properties.name" from the world_geojson.json**

In [63]:
df_without_duplicates = df.drop_duplicates(subset=['location'])
df_without_duplicates
print(len(df_without_duplicates))

244


In [64]:
common_df = df_without_duplicates[df_without_duplicates['location'].isin(df_geo_world['features_properties.name'])]

In [65]:
intersect_locations = common_df['location']

In [66]:
final_output = df.loc[df['location'].isin(intersect_locations.values)]
print(len(final_output))

134470


In [67]:
final_output.to_csv("CSC3007_datasets/owid-covid-data_processed.csv", encoding='utf-8')